# StockTradingEnv


**Características del environment**

StockTradingEnv es un entorno de trading de acciones para OpenAI Gym, donde un agente toma decisiones de compra y venta de acciones. Las características del environment son las siguientes:


**Tipos de estados**

Espacio de Estados:
El espacio de estados es continuo y multidimensional, compuesto por:

*   Posición de apertura de las acciones de los últimos cinco días, escalada entre 0 y 1.
* Máximo precio de las acciones de los últimos cinco días, escalada entre 0 y 1.
* Mínimo precio de las acciones de los últimos cinco días, escalada entre 0 y 1.
* Precio de cierre de las acciones de los últimos cinco días, escalada entre 0 y 1.
* Volumen de las acciones de los últimos cinco días, escalado entre 0 y 1.
* Datos adicionales escalados entre 0 y 1:
  - Saldo actual (balance).
  - Valor neto máximo alcanzado (max_net_worth).
  - Acciones en posesión (shares_held).
  - Base de coste media de las acciones en posesión (cost_basis).
  - Total de acciones vendidas (total_shares_sold).
  - Valor total de las ventas (total_sales_value).

**Tipos de acciones**

Espacio de Acciones:
El espacio de acciones es continuo, con dos dimensiones:

- Tipo de acción (0: Comprar, 1: Vender, 2: Mantener).
- Cantidad de la acción (proporción del saldo o de las acciones en posesión).

**Recompensas**

La recompensa se calcula en función del saldo actual multiplicado por un modificador de retraso, que es proporcional al progreso dentro del episodio.
No hay recompensas positivas o adicionales explícitas por alcanzar ciertos objetivos, pero la recompensa implícita es maximizar el valor neto de la cartera a lo largo del tiempo.

Recompensa media esperada
En StockTradingEnv, el objetivo es que el agente aprenda a maximizar el valor neto de su cartera a lo largo del tiempo mediante decisiones de compra y venta de acciones. La recompensa promedio esperada varía según el rendimiento del agente, pero un agente bien entrenado debería ser capaz de aumentar su valor neto de manera consistente, minimizando pérdidas y aprovechando oportunidades de ganancia.

In [ ]:
!pip install -q stable_baselines3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 42.2 MB/s eta 0:00:00


In [ ]:
import json
import datetime as dt
import random
import json
import gym
from gym import spaces
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback

In [ ]:
# Definición de constantes
MAX_ACCOUNT_BALANCE = 2147483647
MAX_NUM_SHARES = 2147483647
MAX_SHARE_PRICE = 5000
MAX_OPEN_POSITIONS = 5
MAX_STEPS = 20000
INITIAL_ACCOUNT_BALANCE = 10000

class StockTradingEnv(gym.Env):
    """Un entorno de trading de acciones para OpenAI gym"""
    metadata = {'render.modes': ['human']}

    def __init__(self, df):
        super(StockTradingEnv, self).__init__()

        self.df = df
        self.reward_range = (0, MAX_ACCOUNT_BALANCE)

        # Espacio de acciones: [acción, cantidad]
        # acción: 0 = mantener, 1 = comprar, 2 = vender
        # cantidad: 0 a 1 (porcentaje del saldo/acciones en posesión)
        self.action_space = spaces.Box(
            low=np.array([0, 0], dtype=np.float32),
            high=np.array([2, 1], dtype=np.float32),
            dtype=np.float32
        )

        # Espacio de observación: contiene los valores OHLC de los últimos cinco días y otras características
        self.observation_space = spaces.Box(
            low=0, high=1, shape=(6, 6), dtype=np.float32)

    def _next_observation(self):
        # Obtener los puntos de datos de acciones de los últimos 5 días y escalarlos entre 0-1
        frame = np.array([
            self.df.loc[self.current_step: self.current_step + 5, 'Open'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step + 5, 'High'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step + 5, 'Low'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step + 5, 'Close'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step + 5, 'Volume'].values / MAX_NUM_SHARES,
        ], dtype=np.float32)

        # Agregar datos adicionales y escalar cada valor entre 0-1
        obs = np.append(frame, [[
            self.balance / MAX_ACCOUNT_BALANCE,
            self.max_net_worth / MAX_ACCOUNT_BALANCE,
            self.shares_held / MAX_NUM_SHARES,
            self.cost_basis / MAX_SHARE_PRICE,
            self.total_shares_sold / MAX_NUM_SHARES,
            self.total_sales_value / (MAX_NUM_SHARES * MAX_SHARE_PRICE),
        ]], axis=0)

        return obs

    def _take_action(self, action):
        # Escalar las acciones al rango esperado
        action_type = int(action[0])  # 0 = mantener, 1 = comprar, 2 = vender
        amount = action[1]  # Esto ya está en el rango [0, 1]

        # Establecer el precio actual a un precio aleatorio dentro del intervalo de tiempo
        current_price = random.uniform(
            self.df.loc[self.current_step, "Open"], self.df.loc[self.current_step, "Close"])

        if action_type == 1:
            # Comprar amount % del saldo en acciones
            total_possible = int(self.balance / current_price)
            shares_bought = int(total_possible * amount)
            prev_cost = self.cost_basis * self.shares_held
            additional_cost = shares_bought * current_price

            self.balance -= additional_cost
            self.cost_basis = (prev_cost + additional_cost) / (self.shares_held + shares_bought)
            self.shares_held += shares_bought

        elif action_type == 2:
            # Vender amount % de las acciones en posesión
            shares_sold = int(self.shares_held * amount)
            self.balance += shares_sold * current_price
            self.shares_held -= shares_sold
            self.total_shares_sold += shares_sold
            self.total_sales_value += shares_sold * current_price

        self.net_worth = self.balance + self.shares_held * current_price

        if self.net_worth > self.max_net_worth:
            self.max_net_worth = self.net_worth

        if self.shares_held == 0:
            self.cost_basis = 0

    def step(self, action):
        # Ejecutar un paso dentro del entorno
        self._take_action(action)

        self.current_step += 1

        if self.current_step > len(self.df.loc[:, 'Open'].values) - 6:
            self.current_step = 0

        delay_modifier = (self.current_step / MAX_STEPS)

        # Calcular la recompensa basada en el cambio de valor neto
        reward = self.net_worth * delay_modifier
        done = self.net_worth <= 0

        obs = self._next_observation()

        return obs, reward, done, {}

    def reset(self):
        # Reiniciar el estado del entorno a un estado inicial
        self.balance = INITIAL_ACCOUNT_BALANCE
        self.net_worth = INITIAL_ACCOUNT_BALANCE
        self.max_net_worth = INITIAL_ACCOUNT_BALANCE
        self.shares_held = 0
        self.cost_basis = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        # Establecer el paso actual en un punto aleatorio dentro del marco de datos
        self.current_step = random.randint(0, len(self.df.loc[:, 'Open'].values) - 6)

        return self._next_observation()

    def render(self, mode='human', close=False):
        profit = self.net_worth - INITIAL_ACCOUNT_BALANCE

        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance}')
        print(f'Shares held: {self.shares_held} (Total sold: {self.total_shares_sold})')
        print(f'Avg cost for held shares: {self.cost_basis} (Total sales value: {self.total_sales_value})')
        print(f'Net worth: {self.net_worth} (Max net worth: {self.max_net_worth})')
        print(f'Profit: {profit}')

        plt.figure(figsize=(10, 6))
        plt.subplot(2, 1, 1)
        plt.plot(self.df.loc[:self.current_step, 'Close'], label='Close Price')
        plt.title('Stock Price Over Time')
        plt.xlabel('Time')
        plt.ylabel('Price')
        plt.legend()

        plt.subplot(2, 1, 2)
        plt.plot([INITIAL_ACCOUNT_BALANCE] + [self.net_worth], label='Net Worth')
        plt.title('Net Worth Over Time')
        plt.xlabel('Time')
        plt.ylabel('Net Worth')
        plt.legend()

        plt.tight_layout()
        plt.show()



Trabajamos con datos históricos de acciones de Apple (AAPL).

---


El DataFrame contiene un total de 5255 filas y 8 columnas.
Las columnas incluyen información sobre el índice, la fecha, los precios de apertura, alto, bajo y cierre, así como el volumen de transacciones.
Los datos están ordenados cronológicamente por fecha.
Estadísticas Resumidas:

Los precios de apertura, alto, bajo y cierre tienen una amplia gama de valores, con desviaciones estándar significativas, lo que indica una gran variabilidad en los precios a lo largo del tiempo.
El volumen de transacciones también varía considerablemente, con un rango que va desde cientos de miles hasta casi 200 millones.
Tipos de Datos:
La mayoría de las columnas contienen datos numéricos, con la excepción de la columna "Date", que parece contener fechas en formato de objeto.
Es importante convertir la columna "Date" a un tipo de dato de fecha y hora para facilitar su manipulación y análisis temporal.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Master IA^3/Reinforcement Learning/Proyecto RL/Custom Trading Environment/Data/AAPL.csv')
df = df.sort_values('Date').reset_index(drop=True)
df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']].dropna()


In [ ]:
# Verificar las primeras filas del dataframe
print("Primeras filas del dataframe:")
print(df.head())

# Obtener estadísticas resumidas del dataframe
print("\nEstadísticas resumidas del dataframe:")
print(df.describe())

# Verificar el tamaño del dataframe
print("\nTamaño del dataframe:")
print(df.shape)

# Verificar el tipo de datos en cada columna
print("\nTipos de datos en cada columna:")
print(df.dtypes)

Primeras filas del dataframe:
   index  Unnamed: 0        Date   Open   High    Low  Close      Volume
0      0           0  1998-01-02  13.63  16.25  13.50  16.25   6411700.0
1      1           1  1998-01-05  16.50  16.56  15.19  15.88   5820300.0
2      2           2  1998-01-06  15.94  20.00  14.75  18.94  16182800.0
3      3           3  1998-01-07  18.81  19.00  17.31  17.50   9300200.0
4      4           4  1998-01-08  17.44  18.62  16.94  18.19   6910900.0

Estadísticas resumidas del dataframe:
             index   Unnamed: 0         Open         High          Low  \
count  5255.000000  5255.000000  5255.000000  5255.000000  5255.000000   
mean   2627.000000  2627.000000   158.335855   160.109968   156.388133   
std    1517.132163  1517.132163   161.309153   162.588024   159.766714   
min       0.000000     0.000000    12.990000    13.190000    12.720000   
25%    1313.500000  1313.500000    37.940000    38.580000    37.160000   
50%    2627.000000  2627.000000   106.370000   10

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Definir el entorno
# env = DummyVecEnv([lambda: Monitor(StockTradingEnv(df))])

#  # Inicializar el modelo
# model = PPO("MlpPolicy", env, verbose=1)

#  # Crear un callback de evaluación
# eval_env = DummyVecEnv([lambda: StockTradingEnv(df)])
# eval_callback = EvalCallback(eval_env, best_model_save_path='./logs/',
#                               log_path='./logs/', eval_freq=5000,
#                               deterministic=True, render=False)

#  # Entrenar el modelo
# model.learn(total_timesteps=10000, callback=eval_callback)

In [ ]:
# # Entrenamiento del modelo
# model.learn(total_timesteps=20000, callback=eval_callback)

# total_episodes = 100
# total_reward = 0
# total_trades = 0

# for episode in range(total_episodes):
#     obs = eval_env.reset()
#     done = False
#     episode_reward = 0
#     episode_trades = 0

#     while not done:
#         action, _ = model.predict(obs)
#         obs, rewards, done, info = eval_env.step(action)
#         episode_reward += np.sum(rewards)
#         episode_trades += np.sum([1 for r in rewards if r != 0])

#     total_reward += episode_reward
#     total_trades += episode_trades

#     # Renderizar y guardar la gráfica al final de cada episodio
#     eval_env.envs[0].render()

# avg_reward = total_reward / total_episodes
# avg_trades = total_trades / total_episodes

# print(f"Media de recompensa en {total_episodes} episodios: {avg_reward}")
# print(f"Número medio de operaciones en {total_episodes} episodios: {avg_trades}")